Consider a location family given by a density $f_{\mu}$ parameterized by $\mu \in \mathbb{R}$, given by

$$
f_\mu(x) = \frac12 e^{-\lvert x - \mu \rvert}, x \in \mathbb{R}.
$$

This is known as the [Laplace](https://en.wikipedia.org/wiki/Laplace_distribution) or Double Exponential family with location parameter $\mu$.

Given an i.i.d. random sample $X_1, X_2, \dotsc, X_n$ from $f_{\mu}$, where $\mu$ is unknown, we are interested in obtaining a 95% condfidence interval for $\mu.$

## Approximate Normal Interval

It is easy to verify that $E(X_1) = \mu$ and $Var(X_1) = 2$.
Let $\bar{X}$ be the sample mean of $X_1, X_2, \dotsc, X_n.$ Then, by the Central Limit Theorem, we know that the approximate distribution of $\bar{X} - \mu$ is

$$
\bar{X} - \mu \approx N(0, 2 / n).
$$

In other words, since $P(-1.96 < N(0, 1) < 1.96) \approx 0.95$, we have

$$
P(1.96 \sqrt{2/n} < \bar{X} - \mu < 1.96 \sqrt{2/n}) \approx 0.95 \implies P(\bar{X} - 1.96 \sqrt{2/n} < \mu < \bar{X} + 1.96 \sqrt{2/n}) \approx 0.95
$$

This leads to the following function to obtain an approximate 95% confidence interval for $\mu$.

In [3]:
laplace_ci_mean <- function(x)
{
    n <- length(x)
    xbar <- mean(x)
    c(lower = xbar - 1.96 * sqrt(2/n), upper = xbar + 1.96 * sqrt(2/n))
}

We can simulate such intervals many times to estimate two relevant characteristics: the coverage probability (which should be approximately 95%) and the average length (shorter intervals are more useful). Both of these can depend on the sample size $n$ (the Normal approximation may not be ).

We can simulate observations from the Laplace distribution by first noting that the CDF $F$ is given by

$$
F_{\mu}(x) = \begin{cases}
\frac12 e^{x-\mu} & \text{if } x < \mu \cr
1 - \frac12 e^{-(x-\mu)} & \text{if } x \geq \mu
\end{cases},
$$

and then recall that if $U \sim U(0, 1)$, then $F_{\mu}^{-1}(U)$ has CDF $F_{\mu}$. Here,

$$
F_{\mu}^{-1}(u) = \begin{cases}
\mu + \log 2u & \text{if } u < \frac12 \cr
\mu - \log 2 (1-u) & \text{if } u \geq \frac12
\end{cases}.
$$

In [4]:
rlaplace <- function(n, mu = 0)
{
    u <- runif(n)
    ifelse(u < 0.5, mu + log(2*u), mu - log(2 * (1-u)))
}

We can now use this to check whether the target coverage probability is approximately satisfied.

In [6]:
cimean51 <- replicate(10000, laplace_ci_mean(rlaplace(51, mu = 5))) |> t() |> as.data.frame()
cimean501 <- replicate(10000, laplace_ci_mean(rlaplace(501, mu = 5))) |> t() |> as.data.frame()
print(head(cimean51))

     lower    upper
1 4.322367 5.098642
2 4.237200 5.013476
3 4.232194 5.008469
4 4.477667 5.253943
5 4.585784 5.362060
6 4.806288 5.582564


In [7]:
with(cimean51, sum(lower < 5 & upper > 5)) / nrow(cimean51)

[1] 0.9527

In [8]:
with(cimean501, sum(lower < 5 & upper > 5)) / nrow(cimean501)

[1] 0.9518

We can also compute the average length of the confidence intervals as follows.

In [9]:
print(with(cimean51, mean(upper - lower)))
print(with(cimean501, mean(upper - lower)))

[1] 0.7762757
[1] 0.247675


# Your assignment

The confidence interval outlined above is based on the sample mean, using the fact that the function $\bar{X} - \mu$ is a _pivotal quantity_ whose distribution is approximately known.

Your task is to obtain a similar _exact_ confidence interval based on the sample median $\tilde{X}$. It is intuitively clear that $\tilde{X} - \mu$ is also a pivotal quantity, whose distribution does not depend on $\mu$. If we can find a constant $c$ such that

$$
P(-c < \tilde{X} - \mu < c) = 0.95,
$$

then we can repeat the process above to obtain a 95% confidence interval for $\mu$. To obtain the value of $c$, we need to find the distribution of $\tilde{X} - \mu$. You can do so using the following results:

__Result 1__: If $U_1, \dotsc, U_n \sim$ i.i.d. $U(0, 1)$, where $n = 2k + 1$ is odd, then the sample median $\tilde{U} \sim Beta(k, k)$.

__Result 2__: By invariance of the median under monotone transformations, $F_{\mu}^{-1}(\tilde{U})$ and $\tilde{X}$ have the same distribution.

## Instructions

Using the results above, compute $c_n$ as a function of $n$ such that

$$
P(-c_n < \tilde{X} - \mu < c_n) = 0.95,
$$

for $n$ odd. Use it to write a function named `laplace_ci_median` to compute a confidence interval for $\mu$.

In [12]:
laplace_ci_median <- function(x)
{
    n <- length(x)
    xmed <- median(x)
    # begin your code here
    c_n <- ...
    # end your code here
    c(lower = xmed - c_n, upper = xmed + c_n)
}

Once you are satisfied with your solution, run the following cells to verify the coverage probability and average length of the resulting confidence intervals.

In [ ]:
cimed51 <- replicate(10000, laplace_ci_median(rlaplace(51, mu = 5))) |> t() |> as.data.frame()
cimed501 <- replicate(10000, laplace_ci_median(rlaplace(501, mu = 5))) |> t() |> as.data.frame()
print(with(cimed51, sum(lower < 5 & upper > 5)) / nrow(cimed51))
print(with(cimed501, sum(lower < 5 & upper > 5)) / nrow(cimed501))
print(with(cimed51, mean(upper - lower)))
print(with(cimed501, mean(upper - lower)))

Based on the above results, which confidence interval would you prefer?

Instructions for submission will be provided in due course.